# Word embedding and one-hot encoding







## One-hot encoding

> One-hot encoding is the process of turning categorical factors into a numerical structure that machine learning algorithms can readily process. It functions by representing each category in a feature as a binary vector of 1s and 0s, with the vector's size equivalent to the number of potential categories.

In [1]:
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']

### One-hot integer encoding

In [2]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(np.array(data))

print(data)
print(integer_encoded)

['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
[0 0 2 0 1 1 2 0 2 1]


### One-hot binary encoding

In [3]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_data = one_hot_encoder.fit_transform(integer_encoded)

print(data)
print(onehot_encoded_data)

['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Problem 1
What are the limitations of one-hot encoding?

Based on the one-hot encoding example we have an issue that there is no reason why one word corresponds to a bigger number than another. To fix this problem, we use a one-hot encoding vector, initializing a vector of zeros of dimension V  and then putting a 1 in the index corresponding to the word we represent.

# **Conlusion**
- The **Cons** of one-hot integers: can't explain why one word corresponds to a bigger number than another.
- The **Cons** of one-hot vectors: huge and encode no meaning


## Word embedding

ELI5 for word embeddings
> The word embeddings can be thought of as a child’s understanding of the words. Initially, the word embeddings are randomly initialized and they don’t make any sense, just like the baby has no understanding of different words. It’s only after the model has started getting trained, the word vectors/embeddings start to capture the meaning of the words, just like the baby hears and learns different words."

In [4]:
import torch
from torch import nn
from torch.nn import functional as F

In [5]:
import pandas as pd

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

### Unigram transformation

In [6]:
from nltk import ngrams
from typing import List

def ngrams_transform(document: List[str],
                     n_gram: int) -> List[str]:
    """
    N-grams transformations for a given text

    Args:
    document (List[str]) -- The document to-be-processed
    n_gram   (int)       -- Number of grams

    Returns:
    A list of string after n-grams processed
    """
    all_sentence =[]
    ### START YOUR CODE HERE ###
    for i in range(len(document)- n_gram +1):
      splitted_sentence = ngrams(document[i].split(), n_gram)


      # Connect splitted results into 1 list
      n_grams_list = [' '.join(n_gram) for n_gram in splitted_sentence]

      all_sentence +=n_grams_list

    return all_sentence

    ### END YOUR CODE HERE ###

In [7]:
n_grams_list = ngrams_transform(corpus,
                                n_gram=1)
n_grams_list

['This',
 'is',
 'the',
 'first',
 'document.',
 'This',
 'document',
 'is',
 'the',
 'second',
 'document.',
 'And',
 'this',
 'is',
 'the',
 'third',
 'one.',
 'Is',
 'this',
 'the',
 'first',
 'document?']

In [8]:
# Integer label for the given corpus
label_encoder = LabelEncoder()
corpus_vector = label_encoder.fit_transform(np.array(n_grams_list))

# Tensorize the input vector
example_text_tensor = torch.Tensor(corpus_vector).to(dtype=torch.long)
print(f"Example text tensor: {example_text_tensor}")
print(f"Shape of example text tensor: {example_text_tensor.shape}")

Example text tensor: tensor([ 2,  7, 10,  6,  4,  2,  3,  7, 10,  9,  4,  0, 12,  7, 10, 11,  8,  1,
        12, 10,  6,  5])
Shape of example text tensor: torch.Size([22])


### Create an example for embedding function to map from a word dimension to a lower dimensional space

In [9]:
num_vocab = 22 # number of vocabulary
num_dimension = 50 # dimensional embeddings

# Declare the mapping function
example_embedding_function = nn.Embedding(num_vocab, num_dimension)

In [10]:
example_output_tensor = example_embedding_function(example_text_tensor)
print(f"Embedding shape: {example_output_tensor.shape}")

Embedding shape: torch.Size([22, 50])


# Word2vec


* Word2vec is a **class of models** that represents a word in a large text corpus as a vector in n-dimensional space(or n-dimensional feature space) bringing similar words closer to each other.



* Word2vec is a simple yet popular model to construct representating embedding for words from a representation space to a much lower dimensional space (compared to the respective number of words in a dictionary).



* Word2Vec has two neural network-based variants, which are:

    * Continuous Bag of Words (CBOW)
    * Skip-gram.
![](https://kavita-ganesan.com/wp-content/uploads/skipgram-vs-cbow-continuous-bag-of-words-word2vec-word-representation-2048x1075.png)


## Continuous Bag of words (CBOW)

* The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep learning. It is a model that tries to predict words given the context of a few words before and a few words after the target word. This is distinct from language modeling, since CBOW is not sequential and does not have to be probabilistic. Typically, CBOW is used to quickly train word embeddings, and these embeddings are used to initialize the embeddings of some more complicated model. Usually, this is referred to as pretraining embeddings. It almost always helps performance a couple of percent.

* CBOW is modelled as follows:
    * Given a target word $w_i$ and an $N$ context window on each side, $w_{i-1}, \cdots, w_{i-N}$ and $w_{i+1},\cdots, w_{i+N}$, referring to all context words collectively as $C$.

    * CBOW tries to minimize the objective function:

$$
-\log p(w_i|C) = -\log\text{Softmax}\left(A\left(\sum_{w\in C}q_w\right)+b\right)
$$

where $q_w$ is the embedding of word $w$.

In [11]:
# N = 2 according to the definition
CONTEXT_SIZE = 2

corpus = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

corpus = corpus.split()
len(corpus)

62

### Create an integer mapping

In [12]:
vocab = set(corpus)
vocab_size = len(vocab)

# Integer word mapping
word_to_idx = {word: i for i, word in enumerate(vocab)}
word_to_idx

{'of': 0,
 'directed': 1,
 'study': 2,
 'is': 3,
 'other': 4,
 'spells.': 5,
 'conjure': 6,
 'rules': 7,
 'direct': 8,
 'a': 9,
 'about': 10,
 'computers.': 11,
 'In': 12,
 'computer': 13,
 'We': 14,
 'evolution': 15,
 'processes': 16,
 'program.': 17,
 'to': 18,
 'The': 19,
 'manipulate': 20,
 'Computational': 21,
 'evolve,': 22,
 'with': 23,
 'process': 24,
 'we': 25,
 'things': 26,
 'create': 27,
 'programs': 28,
 'process.': 29,
 'processes.': 30,
 'beings': 31,
 'computational': 32,
 'abstract': 33,
 'our': 34,
 'by': 35,
 'pattern': 36,
 'called': 37,
 'inhabit': 38,
 'are': 39,
 'the': 40,
 'that': 41,
 'effect,': 42,
 'data.': 43,
 'spirits': 44,
 'they': 45,
 'idea': 46,
 'People': 47,
 'As': 48}

### Build context according to the given corpus

In [13]:
data = []

for i in range(CONTEXT_SIZE, len(corpus) - CONTEXT_SIZE):
    context = (
        [corpus[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [corpus[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = corpus[i]
    data.append((context, target))

data

[(['are', 'We', 'to', 'study'], 'about'),
 (['about', 'are', 'study', 'the'], 'to'),
 (['to', 'about', 'the', 'idea'], 'study'),
 (['study', 'to', 'idea', 'of'], 'the'),
 (['the', 'study', 'of', 'a'], 'idea'),
 (['idea', 'the', 'a', 'computational'], 'of'),
 (['of', 'idea', 'computational', 'process.'], 'a'),
 (['a', 'of', 'process.', 'Computational'], 'computational'),
 (['computational', 'a', 'Computational', 'processes'], 'process.'),
 (['process.', 'computational', 'processes', 'are'], 'Computational'),
 (['Computational', 'process.', 'are', 'abstract'], 'processes'),
 (['processes', 'Computational', 'abstract', 'beings'], 'are'),
 (['are', 'processes', 'beings', 'that'], 'abstract'),
 (['abstract', 'are', 'that', 'inhabit'], 'beings'),
 (['beings', 'abstract', 'inhabit', 'computers.'], 'that'),
 (['that', 'beings', 'computers.', 'As'], 'inhabit'),
 (['inhabit', 'that', 'As', 'they'], 'computers.'),
 (['computers.', 'inhabit', 'they', 'evolve,'], 'As'),
 (['As', 'computers.', 'evol

### Problem 2
Name at least 2 limitations at this context construction step? Explain your answers.


1.   Limited Context Window
- On this case, this captures a very narrow slice of the surrounding information. In natural language, meaning can be derived from a broader context, including more distant words and sentence structure.
2.   Neglects Word Order and Sentence Structure
- This approach doesn't consider the inherent order of words in a sentence, which plays a crucial role in conveying meaning. Additionally, it disregards grammatical relationships that can provide valuable context clues.



### Vectorize context

In [14]:
import torch

def make_context_vector(context: List[str], word_to_idx: dict) -> torch.Tensor:
  """
  Converts a list of words (context) into a PyTorch tensor of integer indices.

  Args:
      context (List[str]): A list of words representing the context.
      word_to_idx (dict): A dictionary mapping words to their integer indices.

  Returns:
      torch.Tensor: A PyTorch tensor containing the integer indices of the words in the context.
  """

  # Look up word indices in the dictionary
  indices = [word_to_idx.get(word, 0) for word in context]  # Handle unknown words with index 0

  # Convert the list of indices to a PyTorch tensor
  return torch.tensor(indices, dtype=torch.long)


In [15]:
# Functional test
print("Example sample: ", data[0][0])
make_context_vector(data[0][0], word_to_idx)

Example sample:  ['are', 'We', 'to', 'study']


tensor([39, 14, 18,  2])

### CBOW model implementation

In [16]:
class CBOW(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 embed_dim: int) -> None:
        """
        Model constructor
        """
        super().__init__()

        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

        self.embedding_layer = nn.Embedding(vocab_size, embed_dim)
        self.linear_layer = nn.Linear(embed_dim, vocab_size)

        # Neural weight initialization
        nn.init.xavier_normal_(self.embedding_layer.weight)
        nn.init.xavier_normal_(self.linear_layer.weight)

    def forward(self, inputs):
        """
        Function to conduct forward passing
        """
        embedding = self.embedding_layer(inputs)
        embedding = torch.sum(embedding, dim=1)
        output = self.linear_layer(embedding)
        output_softmax = F.log_softmax(output, dim=1)
        return output_softmax

In [17]:
cbow_model = CBOW(vocab_size=vocab_size,
                  embed_dim=10)

# Enable gradient for model training
cbow_model.train()
cbow_model

CBOW(
  (embedding_layer): Embedding(49, 10)
  (linear_layer): Linear(in_features=10, out_features=49, bias=True)
)

### Train

#### Hyperparameters and training configuration

In [18]:
num_epochs: int = 5
learning_rate: float = 5e-2
optimizer: torch.optim = torch.optim.Adam(cbow_model.parameters(),
                                          lr=learning_rate)

loss_function = nn.NLLLoss()

#### Training phase

In [19]:
for epoch in range(1, num_epochs + 1):
    print(f"#Epoch {epoch}/{num_epochs}")

    # Construct input and target tensor
    inputs, targets = torch.tensor(make_context_vector(data[0][0], word_to_idx)), torch.tensor(word_to_idx[data[0][1]])
    inputs = inputs.unsqueeze(0)
    targets = targets.unsqueeze(0)

    # Join whole data into 1 tensor set
    for idx in range(1, len(data)):
        input_tensor = torch.tensor(make_context_vector(data[idx][0], word_to_idx)).unsqueeze(0)
        target_tensor = torch.tensor(word_to_idx[data[idx][1]]).unsqueeze(0)
        torch.cat((inputs, input_tensor), 0)
        torch.cat((targets, target_tensor), 0)

    # Zero out the gradients from the old instance to avoid tensor accumulation
    cbow_model.zero_grad()

    # Forward passing
    log_probabilities = cbow_model(inputs)

    # Evaluate loss
    loss = loss_function(log_probabilities, targets)

    # Backpropagation
    loss.backward()

    # Update the gradient according to the optimization algorithm
    optimizer.step()

    # Get loss values
    epoch_loss = loss.item()
    print("Loss:", epoch_loss)

#Epoch 1/5
Loss: 4.471459865570068
#Epoch 2/5
Loss: 3.633890151977539
#Epoch 3/5
Loss: 2.8992085456848145
#Epoch 4/5
Loss: 2.156766653060913
#Epoch 5/5
Loss: 1.3504527807235718


<ipython-input-19-560c3a59b815>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs, targets = torch.tensor(make_context_vector(data[0][0], word_to_idx)), torch.tensor(word_to_idx[data[0][1]])
<ipython-input-19-560c3a59b815>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(make_context_vector(data[idx][0], word_to_idx)).unsqueeze(0)


#### Inference

In [21]:
with torch.no_grad(): # No gradient update in inference
    context = ['In', 'processes.', 'we', 'conjure']

    # Vectorize input from text to numeric type
    input_tensor = torch.tensor(make_context_vector(context, word_to_idx)).unsqueeze(0)

    # Model makes prediction
    output_tensor = cbow_model(input_tensor)

    # Get the item id with the highest probability
    prediction = torch.argmax(output_tensor).detach().tolist()

    # Query the respective word from the given item id
    key_list = list(word_to_idx.keys())
    prediction = key_list[prediction]

    print("Context:", context)
    print("Prediction:", prediction)

Context: ['In', 'processes.', 'we', 'conjure']
Prediction: is


<ipython-input-21-149b12a03e6d>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(make_context_vector(context, word_to_idx)).unsqueeze(0)


## Skip-gram

<center>
<img src="https://machinelearningcoban.com/tabml_book/_images/word2vec2.png">
</center>

- Skip gram is based on the distributional hypothesis where words with similar distribution is considered to have similar meanings. Researchers of skip gram suggested a model with less parameters along with the novel methods to make optimization step more efficient.

- Vanilla SkipGram model:

<center>
<img src="https://d3i71xaburhd42.cloudfront.net/a1d083c872e848787cb572a73d97f2c24947a374/5-Figure1-1.png" scale=70%>
</center>

- Main idea is to optimize model so that if it is queried with a word, it should correctly guess all the context (context = 2 in the figure) words. That is,
$$
y=\sigma(Ux)
$$
    - where $x$, $y$ are one-hot encoded word vector, $U$ is the embedding matrix, and $\sigma(\cdot)$ is the softmax function.

With the same dataset, training set for skip gram can be much larger than that of NPLM since it can have $2c$ samples $\left(w_t:w_{t-c}, ...,w_t:w_{t-1},w_t:w_{t+1},...,w_{t+c}\right)$ while other n-gram based models have one $\left((w_{t-c},...w_{t-1},w_{t+1},...,w_{t+c}):w_t\right)$.

In [22]:
corpus = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

In [23]:
class SkipGramModel(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 embed_dim: int) -> None:
        """
        Model construction
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

        ### START YOUR CODE HERE ###
        # Declare embedding function u and v
        # with given vocab size and embed dim using nn.Embedding
        self.v_embedding_layer = nn.Embedding(self.vocab_size, self.embed_dim)
        self.u_embedding_layer = nn.Embedding(self.vocab_size, self.embed_dim)

        # Network weight initialization with Xavier initialization


        ### END YOUR CODE HERE ###

    def forward(self, center_words, context):
        """
        Function to perform forward passing
        """
        v_embedding = self.v_embedding_layer(center_words)
        u_embedding = self.u_embedding_layer(context)

        score = torch.mul(v_embedding, u_embedding)
        score = torch.sum(score, dim=1)
        log_score = F.logsigmoid(score)
        return log_score

In [24]:
skipgram_model = SkipGramModel(vocab_size=vocab_size,
                               embed_dim=128)

skipgram_model.train()
skipgram_model

SkipGramModel(
  (v_embedding_layer): Embedding(49, 128)
  (u_embedding_layer): Embedding(49, 128)
)

### Prepare training data to match the format of SkipGram model

In [25]:
def gather_training_data(corpus,
                         word_to_idx: dict,
                         context_size: int):
    """
    This function is to transform the given corpus
    into the correct format for SkipGram to serve as its input
    """

    training_data = []
    all_vocab_indices = list(range(len(word_to_idx)))

    split_text = corpus.split('\n')

    # For each sentence
    for sentence in split_text:
        indices = []
        indices = [word_to_idx[word] for word in sentence.split(' ')]

        # For each word treated as center word
        for center_word_pos in range(len(indices)):

            # For each window  position
            for w in range(-context_size, context_size+1):
                context_word_pos = center_word_pos + w

                # Make sure we dont jump out of the sentence
                if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                    continue

                context_word_idx = indices[context_word_pos]
                center_word_idx  = indices[center_word_pos]

                # Same words might be present in the close vicinity of each other. we want to avoid such cases
                if center_word_idx == context_word_idx:
                    continue

                training_data.append([center_word_idx, context_word_idx])

    return training_data

In [26]:
training_data = gather_training_data(corpus,
                                     word_to_idx,
                                     context_size=2)
training_data = torch.tensor(training_data).to(dtype=torch.long)
training_data.shape

torch.Size([212, 2])

### Hyperparamters and training configuration

In [27]:
num_epochs: int = 200
learning_rate: float = 5e-1
optimizer: torch.optim = torch.optim.SGD(skipgram_model.parameters(),
                                          lr=learning_rate)

### Training phase

In [28]:
for epoch in range(num_epochs + 1):
    """
    Adapt the given CBOW training code for SkipGram
    Following by the instruction comments, or you could do it on your own ;)
    """
    ### START YOUR CODE HERE ###

    # Construct input and target tensor
    skipgram_model.zero_grad()

    # Zero out the gradients from the old instance to avoid tensor accumulation


    # Forward passing
    logsoftmax_prediction = skipgram_model(inputs, targets)

    # Evaluate loss (Negative log likelihood)
    loss = torch.mean(-1 * logsoftmax_prediction)

    # Backpropagation
    loss.backward()

    # Update the gradient according to the optimization algorithm
    optimizer.step()

    # Get loss values
    epoch_loss = loss.item()

    # Log result
    if epoch % 50 == 0:
        print(f"#Epoch {epoch}/{num_epochs}")
        print("Loss:", epoch_loss)

    ### END YOUR CODE HERE ###

#Epoch 0/200
Loss: 0.8112345933914185
#Epoch 50/200
Loss: 0.5353466868400574
#Epoch 100/200
Loss: 0.41822782158851624
#Epoch 150/200
Loss: 0.3497826159000397
#Epoch 200/200
Loss: 0.30101457238197327


### Inference

In [29]:
with torch.no_grad():
    context = ['we']

    ### START YOUR CODE HERE ###
    # Based on the given inference code in the previous section, training code and the context
    # Implement the inference flow from the given context to an output word
    # Vectorize input from text to numeric type
    input_tensor = torch.tensor(make_context_vector(context, word_to_idx)).unsqueeze(0)

    # Model makes prediction
    output_tensor = cbow_model(input_tensor)

    # Get the item id with the highest probability
    prediction = torch.argmax(output_tensor).detach().tolist()

    # Query the respective word from the given item id
    key_list = list(word_to_idx.keys())
    prediction = key_list[prediction]


    ### END YOUR CODE HERE ###
    print("Context:", context)
    print("Prediction:", prediction)

Context: ['we']
Prediction: about


<ipython-input-29-c02b11bf7d7f>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(make_context_vector(context, word_to_idx)).unsqueeze(0)


## Problem 3
What are the differences between CBOW and Skip-gram?



1.  CBOW
- In the CBOW model, the distributed representations of context (or surrounding words) are combined to predict the word in the middle.
- Several times faster to train than the skip-gram, slightly better accuracy for the frequent words.

2.   Skip-gram model
- The distributed representation of the input word is used to predict the context.
- Works well with a small amount of the training data, represents well even rare words or phrases.

